imports

In [ ]:
import tensorflow as tf

from tensorflow.python.framework import constant_op

from tensorflow import keras

single group risk function modified losses

loss = alpha * risk + (1 - alpha) * group_risk

alpha - value between 0 and 1


In [ ]:
@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxGroupRiskBCEv1(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_Equity_BCE', alpha=1.0):

        super().__init__()

        self.alpha = alpha

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        print('{} alpha:\t{}'.format(name,self.alpha))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype) # prevents division by 0 error

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)

        y_pred = tf.keras.backend.reshape(y_pred,(-1,1))


        # group risk calculation

        cal = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal = tf.math.multiply(cat,cal)

        cat_cal = tf.divide((-tf.reduce_sum(cat_cal, axis=0)), n_cat + epsilon_)


        max_cal = tf.reduce_max(cat_cal)


        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)


        return (self.alpha * bce) + ((1-self.alpha) * max_cal)

    def get_config(self):

        return {'name':'MGR_Equity_BCE','alpha': self.alpha}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxGroupRiskBCEv2(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_Equality_BCE', alpha=1.0):

        super().__init__()

        self.alpha = alpha

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        print('{} alpha:\t{}'.format(name,self.alpha))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)

        y_pred = tf.keras.backend.reshape(y_pred,(-1,1))

        # group risk calculation

        cal = tf.math.log(y_pred)

        cat_cal = tf.math.multiply(cat,cal)

        cat_cal = tf.divide((-tf.reduce_sum(cat_cal, axis=0)), n_cat + epsilon_)


        max_cal = tf.reduce_max(cat_cal)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * max_cal)

    def get_config(self):

        return { 'name':'MGR_Equality_BCE','alpha': self.alpha}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class AbSumDiffBCEv1(tf.keras.losses.Loss):

    def __init__(self,  name='ASD_Equity_BCE', alpha=1.0):

        super().__init__()

        self.alpha = alpha

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        print('{} alpha:\t{}'.format(name,self.alpha))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)

        y_pred = tf.keras.backend.reshape(y_pred,(-1,1))

        # group risk calculation

        cal = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal = tf.math.multiply(cat,cal)

        cat_cal = tf.divide((-tf.reduce_sum(cat_cal, axis=0)), n_cat + epsilon_)


        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal, axis=-1), tf.expand_dims(cat_cal, axis=0)))), 2)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * diff_cal)

    def get_config(self):

        return { 'name':'ASD_Equity_BCE','alpha': self.alpha}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class AbSumDiffBCEv2(tf.keras.losses.Loss):

    def __init__(self,  name='ASD_Equality_BCE', alpha=1.0):

        super().__init__()

        self.alpha = alpha

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        print('{} alpha:\t{}'.format(name,self.alpha))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)

        y_pred = tf.keras.backend.reshape(y_pred,(-1,1))

        # group calculation

        cal = tf.math.log(y_pred)

        cat_cal = tf.math.multiply(cat,cal)

        cat_cal = tf.divide((-tf.reduce_sum(cat_cal, axis=0)), n_cat + epsilon_)


        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal, axis=-1), tf.expand_dims(cat_cal, axis=0)))), 2)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * diff_cal)

    def get_config(self):

        return { 'name':'ASD_Equality_BCE','alpha': self.alpha}

combo group risk function modified losses

loss = alpha * risk + (1 - alpha) * (beta * group_risk_1 + (1 - beta) * group_risk_2)

alpha - value between 0 and 1

beta - value between 0 and 1



In [ ]:
@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxGroupRiskBCEv3(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_Equity_Equality_BCE', alpha=1.0, beta=1.0):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        if self.beta < 0 or self.beta > 1:

            self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)

        # 1st group risk calculation

        cal_1 = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal_1 = tf.math.multiply(cat,cal_1)

        cat_cal_1 = tf.divide((-tf.reduce_sum(cat_cal_1, axis=0)), n_cat + epsilon_)


        max_cal_1 = tf.reduce_max(cat_cal_1)

        # 2nd group risk calculation

        cal_2 = tf.math.log(y_pred)

        cat_cal_2 = tf.math.multiply(cat,cal_2)

        cat_cal_2 = tf.divide((-tf.reduce_sum(cat_cal_2, axis=0)), n_cat + epsilon_)


        max_cal_2 = tf.reduce_max(cat_cal_2)

        # combined group risk calculation

        max_cal = (self.beta * max_cal_1) + ((1-self.beta) * max_cal_2)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)


        return (self.alpha * bce) + ((1-self.alpha) * max_cal)

    def get_config(self):

        return { 'name':'MGR_Equity_Equality_BCE','alpha': self.alpha, 'beta': self.beta}

@keras.saving.register_keras_serializable(package='modified_loss_functions')
class AbSumDiffBCEv3(tf.keras.losses.Loss):

    def __init__(self,  name='ASD_Equity_Equality_BCE', alpha=1.0, beta=1.0):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        if self.beta < 0 or self.beta > 1:

            self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # 1st group risk calculation

        cal_1 = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal_1 = tf.math.multiply(cat,cal_1)

        cat_cal_1 = tf.divide((-tf.reduce_sum(cat_cal_1, axis=0)), n_cat + epsilon_)


        diff_cal_1 = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal_1, axis=-1), tf.expand_dims(cat_cal_1, axis=0)))), 2)

        # 2nd group risk calculation

        cal_2 = tf.math.log(y_pred)

        cat_cal_2 = tf.math.multiply(cat,cal_2)

        cat_cal_2 = tf.divide((-tf.reduce_sum(cat_cal_2, axis=0)), n_cat + epsilon_)


        diff_cal_2 = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal_2, axis=-1), tf.expand_dims(cat_cal_2, axis=0)))), 2)


        # combined group risk calculation

        diff_cal = (self.beta * diff_cal_1) + ((1-self.beta) * diff_cal_2)


        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * diff_cal)

    def get_config(self):

        return { 'name':'ASD_Equity_Equality_BCE','alpha': self.alpha, 'beta': self.beta}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxAbSumDiffBCEv1(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_ASD_Equity_BCE', alpha=1.0, beta=1.0):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        if self.beta < 0 or self.beta > 1:

            self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # group risk calculation

        cal = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal = tf.math.multiply(cat,cal)

        cat_cal = tf.divide(-tf.reduce_sum(cat_cal, axis=0), n_cat + epsilon_)

        # 1st group risk calculation

        max_cal = tf.reduce_max(cat_cal)

        # 2nd group risk calculation

        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal, axis=-1), tf.expand_dims(cat_cal, axis=0)))), 2)

        # combined group risk calculation

        grp_cal = (self.beta * max_cal) + ((1-self.beta) * diff_cal)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * grp_cal)

    def get_config(self):

        return { 'name':'MGR_ASD_Equity_BCE','alpha': self.alpha, 'beta': self.beta}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxAbSumDiffBCEv2(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_ASD_Equality_BCE', alpha=1.0, beta=1.0):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        if self.beta < 0 or self.beta > 1:

            self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # group risk calculation

        cal = tf.math.log(y_pred)

        cat_cal = tf.math.multiply(cat,cal)

        cat_cal = tf.divide(-tf.reduce_sum(cat_cal, axis=0), n_cat + epsilon_)

        # 1st group risk calculation

        max_cal = tf.reduce_max(cat_cal)

        # 2nd group risk calculation

        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal, axis=-1), tf.expand_dims(cat_cal, axis=0)))), 2)

        # combined group risk calculation

        grp_cal = (self.beta * max_cal) + ((1-self.beta) * diff_cal)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * grp_cal)

    def get_config(self):

        return {'name':'MGR_ASD_Equality_BCE','alpha': self.alpha, 'beta': self.beta}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxAbSumDiffBCEv3(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_Equity_ASD_Equality_BCE', alpha=1.0, beta=1.0):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        if self.beta < 0 or self.beta > 1:

            self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # 1st group risk calculation

        cal_1 = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal_1 = tf.math.multiply(cat,cal_1)

        cat_cal_1 = tf.divide((-tf.reduce_sum(cat_cal_1, axis=0)), n_cat + epsilon_)


        max_cal = tf.reduce_max(cat_cal_1)


        # 2nd group risk calculation

        cal_2 = tf.math.log(y_pred)

        cat_cal_2 = tf.math.multiply(cat,cal_2)

        cat_cal_2 = tf.divide((-tf.reduce_sum(cat_cal_2, axis=0)), n_cat + epsilon_)


        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal_2, axis=-1), tf.expand_dims(cat_cal_2, axis=0)))), 2)

        # combined group risk calculation

        grp_cal = (self.beta * max_cal) + ((1-self.beta) * diff_cal)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * grp_cal)

    def get_config(self):

        return { 'name':'MGR_Equity_ASD_Equality_BCE','alpha': self.alpha, 'beta': self.beta}


@keras.saving.register_keras_serializable(package='modified_loss_functions')
class MaxAbSumDiffBCEv4(tf.keras.losses.Loss):

    def __init__(self,  name='MGR_Equality_ASD_Equity_BCE', alpha=1.0, beta=1.0):

        super().__init__()

        self.alpha = alpha

        self.beta = beta

        if self.alpha < 0 or self.alpha > 1:

            self.alpha = 1

        if self.beta < 0 or self.beta > 1:

            self.beta = 1

        print('{} alpha:\t{}\tbeta:\t{}'.format(name,self.alpha,self.beta))

    def call(self, y_true, y_pred):

        yT = y_true[:,:1] # y values

        cat = y_true[:,1:] # category values

        n_cat = tf.reduce_sum(cat, axis=0) # number of instances that belong to each category


        epsilon_ = constant_op.constant(tf.keras.backend.epsilon(), y_pred.dtype.base_dtype)

        y_pred = tf.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)


        # 1st group risk calculation

        cal_1 = tf.math.log(y_pred)

        cat_cal_1 = tf.math.multiply(cat,cal_1)

        cat_cal_1 = tf.divide((-tf.reduce_sum(cat_cal_1, axis=0)), n_cat + epsilon_)


        max_cal = tf.reduce_max(cat_cal_1)

        # 2nd group risk calculation

        cal_2 = yT * tf.math.log(y_pred) + (1 - yT) * tf.math.log(1 - y_pred)

        cat_cal_2 = tf.math.multiply(cat,cal_2)

        cat_cal_2 = tf.divide((-tf.reduce_sum(cat_cal_2, axis=0)), n_cat + epsilon_)


        diff_cal = tf.divide(tf.reduce_sum(tf.abs(tf.subtract(tf.expand_dims(cat_cal_2, axis=-1), tf.expand_dims(cat_cal_2, axis=0)))), 2)

        # combined group risk calculation

        grp_cal = (self.beta * max_cal) + ((1-self.beta) * diff_cal)

        # risk calculation

        bce = yT * tf.math.log(y_pred)

        bce += (1 - yT) * tf.math.log(1 - y_pred)

        bce = tf.reduce_mean(-bce)

        return (self.alpha * bce) + ((1-self.alpha) * grp_cal)

    def get_config(self):

        return { 'name':'MGR_Equality_ASD_Equity_BCE','alpha': self.alpha, 'beta': self.beta}